### References
https://towardsdatascience.com/fine-tuning-the-bart-large-model-for-text-summarization-3c69e4c04582

https://github.com/francoisstamant/Fine-tuning-for-text-summarization/blob/main/text_summarization.ipynb

## Abstractive Fine Tuning

In [ ]:
!pip install ohmeow-blurr -q
!pip install bert-score -q

!pip install torch==1.10.1
!pip install fastai==2.5.3
!pip install transformers==4.16.2

In [1]:
import time
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

In [2]:
import torch
import pandas as pd
from fastai.text.all import *
from transformers import *
from blurr.data.all import *
from blurr.modeling.all import *

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1125)>


In [21]:
from rouge_score import rouge_scorer

In [31]:
df = pd.read_csv('extractive_summaries_v2.csv').drop('Unnamed: 0', axis=1)
print(len(df))
old_df = pd.read_csv("first_1000v2.csv")
df = df[~df["title"].isin(old_df["title"])]
df = shuffle(df, random_state = 42).reset_index(drop=True)
df = df[['title', 'ext_summaries', 'cleaned_description']]

# for gold spotify
# df = df[["episode name", "cleaned_description", "ext_summaries"]]

4180


In [4]:
df.head()

,title,ext_summaries,cleaned_description
0,John Bolton Casts Shadow Over Trump Impeachment Defense,"We are one week away from the Iowa caucuses. And this Friday, we're going to be live in Des Moines to break down everything you need to know before the big day. Join us by grabbing a ticket at nprpresents.org. OK. Here's the show. Hey there. It's the NPR POLITICS PODCAST. It is16 p.m. on Monday, January 27.I'm The president's legal team continues its defense of President Trump in the Senate impeachment trial. And what Bolton alleges in this book is that President Trump told him in August of 2019 that Trump wanted to continue to hold around $400 million in aid to Ukraine until the Ukrainia...","As President Trump's legal team continues their case for acquittal, a report in The New York Times about an alleged conversation between Trump and Bolton — contained in a draft of the former national security adviser's book manuscript — could change the equation for some senators who are undecided on calling witnesses.And, Joe Biden and Rudy Giuliani were both discussed at length today as the president's lawyers attempt to reframe and undercut the arguments made by Democratic House impeachment managers.Connect:Subscribe to the NPR Politics Podcast here.Email the show at nprpolitics@npr.org..."
1,Quick Take: The Battle for Control of the GOP,"Hey there. Sam Sanders here popping in to let you know that this episode was recorded before the terrorist attacks in Paris late Friday night. It was a horrible event and one we're going to talk about here on the podcast in an episode soon. But for now, here's this episode. Hey, y'all. It's the NPR POLITICS PODCAST here with a quick take on something really interesting that's been unfolding over the last week or so within the Republican Party. It's a storyline that's been building for some time now. And it was on the main stage last week at the Fox Business Network debate. That night fea...",What does it mean to be a Republican today? The GOP's presidential candidates are fighting to answer that question.
2,Will A New Georgia Law Restrict Voter Access Or Restore Faith In Elections?,"Hi. This is Shawna (ph) from Madison, Wis. And I am getting ready to teach a virtual tap dance class. This podcast was recorded at... 08 p.m. on Monday, March 29. Things may have changed by the time you hear it. All right, enjoy the show. I like hearing that tap sound. I've been curious about taking, like, fitness classes and group things over Zoom. And I have not been able to get myself there, but it sounds like it might be kind of fun. Might sign you up for a tap class - you might be really good at it. Hey there. It's the NPR POLITICS PODCAST.I'm Susan Davis. Republicans say these are n...","A new Georgia law has become the center of the debate over voting rights with President Biden calling it ""Jim Crow in the 21st century."" Republicans argue the law helps restore faith in the electoral process, but civil rights advocates say it disenfranchises voters of color. Plus, Texas Republicans introduce new bills to restrict voter access."
3,Encore Interview: Authors Candice Carty-Williams And Angie Thomas,"Hi, I'm Anjuli. I'm a producer here on IT'S BEEN A MINUTE. Hi, I'm Alex. I'm an editor on the show. So Alex, let's start by actually telling listeners what it takes to make this show. The process actually begins with pitching. It just begins with throwing ideas out there. And then we actually have to take those ideas and book the people we want on the show. We're taping these interviews with people in studio, sometimes going out to them in the field. And then our producers put their headphones on, and they start cutting this massive thing down. And then editors like Alex on our show are a...","On this special episode, Sam Sanders revisits his 2019 conversations with two writers whose books he loved: Candice Carty-Williams, author of 'Queenie,' and Angie Thomas, author of the books, 'On The Come Up' and 'The Hate U Give.' Can

In [5]:
train_len = int(round(4180 * 0.9)) - 1000

train_df = df.iloc[:train_len].reset_index(drop=True)
test_df = df.iloc[train_len:].reset_index(drop=True)

In [7]:
test_df.head()

,title,ext_summaries,cleaned_description
0,Weekly Roundup: December 18th,"And then as it provides updates to its customers, which include U.S. government agencies, big businesses and others, they are ingesting the malware that you seeded a couple of those steps away. The list of affected government agencies at this point in time - we know that the Commerce Department, the Department of Homeland Security, the Department of Defense, the Treasury Department, Postal Service, Energy Department, National Institutes of Health have all been affected in some way, shape or form. Or will this be one of the things for which the administration doesn't act until we have the a...",A major hack of federal government computer systems appears to be state-sponsored espionage. And we talk through Joe Biden's latest cabinet picks.
1,Amid Many Global Crises Biden Calls For Togetherness In First U.N. Address,"Hi. This is Christina (ph) from Austin, Texas. And right now I'm just sitting on my couch with my dog, George (ph). George is very tired. But it's because he and I passed our therapy team test. He's now officially a therapy animal. This podcast was recorded at... 33 p.m. Eastern Time on Tuesday, September 21. Things may have changed by the time you listen to it. But hopefully George wakes up from this nap soon. All right. President Biden delivered his first address to the United Nations General Assembly today, and it comes just weeks after the chaotic withdrawal from Afghanistan. But I ju...","In his first address to the United Nations General Assembly, President Biden emphasized the importance of global cooperation to combat the coronavirus and climate change. And he not so subtly critiqued China and authoritarianism."
2,How The Rat Blew Up,"This is the moment where we're supposed to ask you to show your support for PLANET MONEY by making a donation, but that's not exactly fun. So we made this little promo instead. Do you have economic questions rattling around in your head, driving you nuts, keeping you awake at night? You've tried shouting them out a window. (As character) Why are oil prices negative? But nobody answers. You've tried bringing them up during pillow talk. (As character) Have you ever heard of an inverted yield curve? (As character) Honey, I have a headache. (As character) Maybe another night. Even Siri doesn'...","Unions have been putting giant inflatable rats in front of businesses for years. Now businesses are trying to deflate them, in court. | Subscribe to our weekly newsletter here."
3,The Madness Of March,"The quest for the crown is underway.() Jim, we're going to see some of the best long-armed quick athletes that there is in college basketball. But Ed and his teammates were student athletes. They wanted to file a lawsuit against the NCAA because they said that the NCAA's control of college athletes and how much they could earn violated laws around fair compensation. So the NCAA has thousands of member colleges and universities and hundreds of thousands of student athletes. According to a poll conducted last fall by The Washington Post and ABC News, nearly 2 out of 3 Americans, in fact, thi...","The NCAA men's basketball tournament is going on right now and will bring in hundreds of millions of dollars in revenue. The coaches and commissioners who benefit are overwhelmingly white. The players on the court are MOSTLY black. So what, if anything, are those players owed?"
4,How To Choose A Health Insurance Plan,"When open enrollment for health insurance rolls around, usually in the fall, HR departments and local brokers try to make it fun - balloons, kazoos. Happy open enrollment, everyone - so exciting. Are you having fun yet? No? For real, picking health insurance is work. Before we dive in, let's talk about why health insurance is worth having - in other words, why it's worthwhile to put in all the effort to find a plan and enroll. To begin with, you can think about health insurance like your local fire department. I'm 

In [9]:
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, 
                                                                  model_cls=BartForConditionalGeneration)

In [10]:
hf_arch

'bart'

In [11]:
hf_config

BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "L

In [12]:
hf_tokenizer

PreTrainedTokenizerFast(name_or_path='facebook/bart-large-cnn', vocab_size=50265, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [13]:
hf_model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
   

In [14]:
hf_batch_tfm = HF_Seq2SeqBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model, task='summarization',
text_gen_kwargs={
    # vals are controlling for padding, do not need
#     'max_length': 248,
#  'min_length': 56,
 'do_sample': False,
    # change early stopping to False
 'early_stopping': False,
 'num_beams': 4,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'repetition_penalty': 1.0,
 'bad_words_ids': None,
 'bos_token_id': 0,
 'pad_token_id': 1,
 'eos_token_id': 2,
 'length_penalty': 2.0,
 'no_repeat_ngram_size': 3,
 'encoder_no_repeat_ngram_size': 0,
 'num_return_sequences': 1,
 'decoder_start_token_id': 2,
 'use_cache': True,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'output_attentions': False,
 'output_hidden_states': False,
 'output_scores': False,
 'return_dict_in_generate': False,
 'forced_bos_token_id': 0,
 'forced_eos_token_id': 2,
 'remove_invalid_values': False})

In [15]:
#Prepare data for training
blocks = (HF_Seq2SeqBlock(before_batch_tfm=hf_batch_tfm), noop)

dblock = DataBlock(blocks=blocks, get_x=ColReader('ext_summaries'), get_y=ColReader('cleaned_description'), splitter=RandomSplitter())

In [16]:
dls = dblock.dataloaders(test_df, bs=2)

In [17]:
#Define performance metrics
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]}}

In [18]:
model = HF_BaseModelWrapper(hf_model)
learn_cbs = [HF_BaseModelCallback]
fit_cbs = [HF_Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

learn = Learner(dls, 
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,
                splitter=partial(seq2seq_splitter, arch=hf_arch)).to_fp16()

learn.create_opt() 
learn.freeze()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [19]:
# FOR FINE TUNING
# start = time.time()
# learn.fit_one_cycle(1, lr_max=3e-5, cbs=fit_cbs)
# end = time.time()
# print(end-start)
# ----------------------------------------------------

# FOR RELOADING
new_learn = Learner(dls, 
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,
                splitter=partial(seq2seq_splitter, arch=hf_arch)).to_fp16()
new_learn  = new_learn.load(file = "fine_tuned_bart_cnn_1")

In [20]:
# learn.save("fine_tuned_bart_cnn_1")

## Evaluate Model

In [22]:
train_rouge1s, train_rouge2s, train_rougeLs = [], [], []
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeLsum'], use_stemmer=True)
train_outputs = []

# train_outputs = [learn.blurr_generate(i, early_stopping=False, num_return_sequences=1) for i in train_df['ext_summaries']]
for i in range(len(test_df['ext_summaries'])):
    start = time.time()
    train_outputs += [new_learn.blurr_generate(test_df['ext_summaries'][i], 
                                           early_stopping=False, num_return_sequences=1)]
    scores = scorer.score(train_outputs[-1][0], test_df['cleaned_description'][i])
    train_rouge1s += [scores['rouge1'][2]]
    train_rouge2s += [scores['rouge2'][2]]
    train_rougeLs += [scores['rougeLsum'][2]]
    end = time.time()
    print(i, end-start)

0 14.15987777709961
1 24.91643190383911
2 22.961649894714355
3 27.153764963150024
4 24.191830158233643
5 24.804927110671997
6 23.679725170135498
7 28.837836027145386
8 16.029541015625
9 14.92687702178955
10 17.352323055267334
11 21.58385396003723
12 13.471438884735107
13 19.09633994102478
14 15.627184867858887
15 25.00234889984131
16 15.092352151870728
17 13.286089897155762
18 21.596368074417114
19 11.3197181224823
20 16.546256065368652
21 26.357810258865356
22 15.847020149230957
23 30.381231784820557
24 29.112615823745728
25 33.568498849868774
26 17.10931897163391
27 20.420526027679443
28 21.57655096054077
29 24.779188871383667
30 31.12147617340088
31 17.743190050125122
32 29.525331020355225
33 19.130318880081177
34 21.868350982666016
35 33.795284032821655
36 36.129814863204956
37 17.740372896194458
38 21.93385601043701
39 19.41888999938965
40 16.36072278022766
41 18.999150276184082
42 18.26339888572693
43 25.61358904838562
44 23.944570779800415
45 14.139671802520752
46 22.17011213302

367 17.282117128372192
368 26.628830909729004
369 18.54355525970459
370 27.005949020385742
371 25.220686674118042
372 17.539305925369263
373 27.97062397003174
374 23.22268581390381
375 22.644584894180298
376 18.861665964126587
377 13.459535121917725
378 26.378538131713867
379 25.004909992218018
380 26.115046739578247
381 20.242048025131226
382 21.735896110534668
383 19.050585985183716
384 10.934141874313354
385 23.80578374862671
386 31.867028951644897
387 15.50243091583252
388 18.537184953689575
389 25.926053047180176
390 19.24666714668274
391 22.50014591217041
392 27.078269958496094
393 19.068174123764038
394 16.322293996810913
395 29.82699990272522
396 28.40213894844055
397 17.659571886062622
398 26.775118112564087
399 20.169328212738037
400 25.076535940170288
401 14.333399057388306
402 18.089038133621216
403 20.298736095428467
404 21.24873113632202
405 18.27119016647339
406 11.799903869628906
407 16.188091039657593
408 23.639997959136963
409 25.534321308135986
410 27.443011045455933

In [ ]:
print("hello")

In [25]:
curr_df = test_df.copy()
curr_df["final_summary"] = train_outputs
curr_df["rouge1 final"] = train_rouge1s
curr_df["rouge2 final"] = train_rouge2s
curr_df["rougeLsum final"] = train_rougeLs
# curr_df

In [26]:
# first3500 = pd.read_csv("first_3500.csv", index_col = 0)
# first4000 = pd.concat([first3500, curr_df])
# first4000.to_csv("all_train_final.csv")

curr_df.to_csv("test_final_summaries.csv")

In [30]:
print("Test ROUGE 1: ", np.mean(curr_df["rouge1 final"]))
print("Test ROUGE 2: ", np.mean(curr_df["rouge2 final"]))
print("Test ROUGE LSUM: ", np.mean(curr_df["rougeLsum final"]))

Test ROUGE 1:  0.29926070878928346
Test ROUGE 2:  0.10360578490359791
Test ROUGE LSUM:  0.193881724958022


In [ ]:
# train_df["final_summary"] = train_outputs

In [ ]:
# start = time.time()
# test_outputs = [learn.blurr_generate(i, early_stopping=False, num_return_sequences=1) for i in test_df['ext_summaries']]
# end = time.time()
# print(end-start)
# test_df["final_summary"] = test_outputs

In [ ]:
# from rouge_score import rouge_scorer

# train_rouge1s, train_rouge2s, train_rougeLs = [], [], []
# scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeLsum'], use_stemmer=True)

# for i in range(len(train_outputs)):
#     scores = scorer.score(train_outputs[i][0], df['cleaned_description'][i])
#     train_rouge1s += [scores['rouge1'][2]]
#     train_rouge2s += [scores['rouge2'][2]]
#     train_rougeLs += [scores['rougeLsum'][2]]



In [ ]:
curr_df.to_csv("first_100.csv")

In [ ]:
np.mean(train_rouge1s)

In [ ]:
np.mean(train_rouge2s)

In [ ]:
np.mean(train_rougeLs)

In [ ]:
test_rouge1s, test_rouge2s, test_rougeLs = [], [], []
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeLsum'], use_stemmer=True)

for i in range(len(test_outputs)):
    scores = scorer.score(test_outputs[i][0], df['cleaned_description'][i])
    test_rouge1s += [scores['rouge1'][2]]
    test_rouge2s += [scores['rouge2'][2]]
    test_rougeLs += [scores['rougeLsum'][2]]



In [ ]:
np.mean(test_rouge1s)

In [ ]:
np.mean(test_rouge2s)

In [ ]:
np.mean(test_rougeLs)

# Reload Model

In [ ]:
new_learn = Learner(dls, 
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,
                splitter=partial(seq2seq_splitter, arch=hf_arch)).to_fp16()
new_learn  = new_learn.load(file = "learner_test")

In [ ]:
outputs = [new_learn.blurr_generate(i, early_stopping=False, num_return_sequences=1) for i in df['ext_summaries']]

for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

# ROUGE Scores

In [28]:
final_train_df = pd.read_csv("all_train_final.csv", index_col = 0)
final_test_df = pd.read_csv("test_final.csv", index_col = 0)

In [ ]:
train_rouge1s, train_rouge2s, train_rougeLs = [], [], []
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeLsum'], use_stemmer=True)

for i in range(len(final_test_df)):
    row = final_test_df.iloc[i]
    scores = scorer.score(row["final_summary"][2:-2], row["cleaned_description"])
    train_rouge1s += [scores['rouge1'][2]]
    train_rouge2s += [scores['rouge2'][2]]
    train_rougeLs += [scores['rougeLsum'][2]]

In [ ]:
print("Test ROUGE 1: ", np.mean(train_rouge1s))
print("Test ROUGE 2: ", np.mean(train_rouge2s))
print("Test ROUGE LSUM: ", np.mean(train_rougeLs))

In [32]:
print("Train ROUGE 1: ", np.mean(final_train_df["rouge1 final"]))
print("Train ROUGE 2: ", np.mean(final_train_df["rouge2 final"]))
print("Train ROUGE LSUM: ", np.mean(final_train_df["rougeLsum final"]))

Train ROUGE 1:  0.3057937642173362
Train ROUGE 2:  0.10367278842471121
Train ROUGE LSUM:  0.1952444678881282


In [29]:
print("Test ROUGE 1: ", np.mean(final_test_df["rouge1 final"]))
print("Test ROUGE 2: ", np.mean(final_test_df["rouge2 final"]))
print("Test ROUGE LSUM: ", np.mean(final_test_df["rougeLsum final"]))

Test ROUGE 1:  0.31189312505436156
Test ROUGE 2:  0.11083795388258275
Test ROUGE LSUM:  0.20375157203321168


In [36]:
gold_df = pd.read_csv("gold_final.csv", index_col = 0)
print("Gold ROUGE 1: ", np.mean(gold_df["rouge1 final"]))
print("Gold ROUGE 2: ", np.mean(gold_df["rouge2 final"]))
print("Gold ROUGE LSUM: ", np.mean(gold_df["rougeLsum final"]))

Gold ROUGE 1:  0.23434786987766865
Gold ROUGE 2:  0.0450038137181914
Gold ROUGE LSUM:  0.13869516285085218


In [37]:
baseline_df = pd.read_csv("all_summaries.csv", index_col = 0)

In [38]:
baseline_df.head()

,title,rouge 1,rouge 2,rouge L sum,summary
0,Changes To Texas Voting Rules Worry Older Voters And Those With Disabilities,"Score(precision=0.2972972972972973, recall=0.2894736842105263, fmeasure=0.29333333333333333)","Score(precision=0.0273972602739726, recall=0.02666666666666667, fmeasure=0.027027027027027025)","Score(precision=0.14864864864864866, recall=0.14473684210526316, fmeasure=0.14666666666666667)",This was the first Election Day since sweeping election changes in the state that drastically change how a lot of people cast ballots. Only a couple of counties took advantage of unlimited polling hours and drive-through voting. Voting rights advocates say there is quite obviously going to be discriminatory effects from creating new limits on this type of voting. There was also some concern about provisions in the law that give poll-watchers broader authority.
1,"At The State Of The Union, Biden Wooed Moderates And Slammed Putin","Score(precision=0.3448275862068966, recall=0.16129032258064516, fmeasure=0.2197802197802198)","Score(precision=0.03508771929824561, recall=0.016260162601626018, fmeasure=0.022222222222222223)","Score(precision=0.1724137931034483, recall=0.08064516129032258, fmeasure=0.1098901098901099)","President Biden delivered his first State of the Union address on Tuesday. CNN's John Sutter says it was a very traditional American leadership speech. Sutter: ""Get back to normal was sort of his message, which I thought was interesting"" Biden's poll numbers among independent voters, they're in the toilet. He is profoundly unpopular with people who don't identify with either party. He didn't talk more about voting rights. He did not talk about explicitly passing police reform legislation through Congress. I think the relationships between lawmakers have been particularly nasty. There was a..."
2,"""Awake Not Woke"": How Republicans Are Defining Their Party in 2022","Score(precision=0.20754716981132076, recall=0.2972972972972973, fmeasure=0.24444444444444446)","Score(precision=0.019230769230769232, recall=0.027777777777777776, fmeasure=0.02272727272727273)","Score(precision=0.09433962264150944, recall=0.13513513513513514, fmeasure=0.11111111111111112)","Domenico Montanaro: CPAC is now the Trump show. He says the former president, American president, has been essentially praising Putin. Domenico: It's not necessarily predictive of anything except for the people who are in that room."
3,Ketanji Brown Jackson Is The First Black Woman Nominated To The Supreme Court,"Score(precision=0.4791666666666667, recall=0.19491525423728814, fmeasure=0.27710843373493976)","Score(precision=0.10638297872340426, recall=0.042735042735042736, fmeasure=0.06097560975609756)","Score(precision=0.3125, recall=0.1271186440677966, fmeasure=0.18072289156626506)","Teresa: She's only the second generation in her family who had anybody go to college and that her ancestors were most likely, on both sides, her mother's side and her father's side, slaves. Domenico: A new poll out today is not looking good for President Biden. 56% say his first year in office was a failure, with just 39% saying it was a success. We've seen a decline in suburban voters by double digits in their approval of President Biden in just the last two months. Biden is teaching himself Arabic - showoff. He's also apparently teaching himself how to vote. Biden is in the running to be..."
4,US Responds To Russian Invasion Of Ukraine With Stronger Sanctions,"Score(precision=0.4444444444444444, recall=0.21739130434782608, fmeasure=0.29197080291970806)","Score(precision=0.045454545454545456, recall=0.02197802197802198, fmeasure=0.029629629629629627)","Score(precision=0.2, recall=0.09782608695652174, fmeasure=0.13138686131386862)","Vladimir Putin ignored months of warnings from the U.S., Europe and allies around the world that an invasion will provoke severe consequences for Russia and its people. It's really it's also a challenge by Putin to NATO countries, and that includes the Un